In [ ]:
#Convolutional neural network & gated recurrent unit

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.cross_validation import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

#normalize the data
scaler = Normalizer().fit(X)
X = scaler.transform(X)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [5]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [6]:
lstm_output_size = 70

In [7]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(8, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(GRU(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu", input_shape=(8, 1))`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
checkpointer = callbacks.ModelCheckpoint(filepath="logs/cnn-gru/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('logs/cnn-gru/cnntrainanalysis1.csv',separator=',', append=False)
cnn.fit(X_train, y_train, nb_epoch=1000, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
cnn.save("logs/cnn-gru/cnn_model.hdf5")
loss, accuracy = cnn.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

Train on 514 samples, validate on 254 samples
Epoch 1/1000
514/514 [==============================] - 0s 641us/step - loss: 0.6632 - acc: 0.6401 - val_loss: 0.6404 - val_acc: 0.6614
Epoch 2/1000
514/514 [==============================] - 0s 204us/step - loss: 0.6565 - acc: 0.6459 - val_loss: 0.6398 - val_acc: 0.6614
Epoch 3/1000
514/514 [==============================] - 0s 212us/step - loss: 0.6503 - acc: 0.6459 - val_loss: 0.6399 - val_acc: 0.6614
Epoch 4/1000
514/514 [==============================] - 0s 236us/step - loss: 0.6491 - acc: 0.6459 - val_loss: 0.6387 - val_acc: 0.6614
Epoch 5/1000
514/514 [==============================] - 0s 215us/step - loss: 0.6501 - acc: 0.6459 - val_loss: 0.6379 - val_acc: 0.6614
Epoch 6/1000
514/514 [==============================] - 0s 220us/step - loss: 0.6481 - acc: 0.6459 - val_loss: 0.6379 - val_acc: 0.6614
Epoch 7/1000
514/514 [==============================] - 0s 264us/step - loss: 0.6483 - acc: 0.6459 - val_loss: 0.6374 - val_acc: 0.6614
Ep

514/514 [==============================] - 0s 184us/step - loss: 0.6081 - acc: 0.6595 - val_loss: 0.6216 - val_acc: 0.6575
Epoch 33/1000
514/514 [==============================] - 0s 199us/step - loss: 0.6045 - acc: 0.6479 - val_loss: 0.6313 - val_acc: 0.6535
Epoch 34/1000
514/514 [==============================] - 0s 189us/step - loss: 0.6337 - acc: 0.6440 - val_loss: 0.6066 - val_acc: 0.6772
Epoch 35/1000
514/514 [==============================] - 0s 176us/step - loss: 0.6023 - acc: 0.6712 - val_loss: 0.6065 - val_acc: 0.6654
Epoch 36/1000
514/514 [==============================] - 0s 189us/step - loss: 0.6071 - acc: 0.6634 - val_loss: 0.6097 - val_acc: 0.6732
Epoch 37/1000
514/514 [==============================] - 0s 185us/step - loss: 0.5985 - acc: 0.6634 - val_loss: 0.6078 - val_acc: 0.6614
Epoch 38/1000
514/514 [==============================] - 0s 215us/step - loss: 0.5962 - acc: 0.6770 - val_loss: 0.6104 - val_acc: 0.6850
Epoch 39/1000
514/514 [==============================] 

Epoch 65/1000
514/514 [==============================] - 0s 213us/step - loss: 0.6257 - acc: 0.6537 - val_loss: 0.6063 - val_acc: 0.6811
Epoch 66/1000
514/514 [==============================] - 0s 245us/step - loss: 0.5944 - acc: 0.6576 - val_loss: 0.6156 - val_acc: 0.6850
Epoch 67/1000
514/514 [==============================] - 0s 202us/step - loss: 0.5980 - acc: 0.6712 - val_loss: 0.6143 - val_acc: 0.6772
Epoch 68/1000
514/514 [==============================] - 0s 201us/step - loss: 0.5989 - acc: 0.6634 - val_loss: 0.6068 - val_acc: 0.6732
Epoch 69/1000
514/514 [==============================] - 0s 197us/step - loss: 0.6042 - acc: 0.6615 - val_loss: 0.6029 - val_acc: 0.6732
Epoch 70/1000
514/514 [==============================] - 0s 214us/step - loss: 0.5927 - acc: 0.6595 - val_loss: 0.6126 - val_acc: 0.6850
Epoch 71/1000
514/514 [==============================] - 0s 195us/step - loss: 0.6092 - acc: 0.6732 - val_loss: 0.6045 - val_acc: 0.6732
Epoch 72/1000
514/514 [==================

514/514 [==============================] - 0s 179us/step - loss: 0.5940 - acc: 0.6595 - val_loss: 0.6064 - val_acc: 0.6732
Epoch 97/1000
514/514 [==============================] - 0s 172us/step - loss: 0.5941 - acc: 0.6634 - val_loss: 0.6103 - val_acc: 0.6772
Epoch 98/1000
514/514 [==============================] - 0s 171us/step - loss: 0.5908 - acc: 0.6673 - val_loss: 0.6044 - val_acc: 0.6929
Epoch 99/1000
514/514 [==============================] - 0s 186us/step - loss: 0.5890 - acc: 0.6809 - val_loss: 0.6051 - val_acc: 0.6811
Epoch 100/1000
514/514 [==============================] - 0s 177us/step - loss: 0.5887 - acc: 0.6576 - val_loss: 0.6250 - val_acc: 0.6496
Epoch 101/1000
514/514 [==============================] - 0s 185us/step - loss: 0.5991 - acc: 0.6790 - val_loss: 0.6025 - val_acc: 0.6772
Epoch 102/1000
514/514 [==============================] - 0s 177us/step - loss: 0.5871 - acc: 0.6829 - val_loss: 0.6054 - val_acc: 0.6772
Epoch 103/1000
514/514 [============================

514/514 [==============================] - 0s 365us/step - loss: 0.5792 - acc: 0.6907 - val_loss: 0.6055 - val_acc: 0.6969
Epoch 129/1000
514/514 [==============================] - 0s 296us/step - loss: 0.5788 - acc: 0.6693 - val_loss: 0.6045 - val_acc: 0.6929
Epoch 130/1000
514/514 [==============================] - 0s 303us/step - loss: 0.5746 - acc: 0.7043 - val_loss: 0.6120 - val_acc: 0.6535
Epoch 131/1000
514/514 [==============================] - 0s 327us/step - loss: 0.5702 - acc: 0.6848 - val_loss: 0.6530 - val_acc: 0.6811
Epoch 132/1000
514/514 [==============================] - 0s 289us/step - loss: 0.6037 - acc: 0.6868 - val_loss: 0.6148 - val_acc: 0.6614
Epoch 133/1000
514/514 [==============================] - 0s 322us/step - loss: 0.5837 - acc: 0.6809 - val_loss: 0.6106 - val_acc: 0.6811
Epoch 134/1000
514/514 [==============================] - 0s 454us/step - loss: 0.5814 - acc: 0.7121 - val_loss: 0.6149 - val_acc: 0.6654
Epoch 135/1000
514/514 [=========================

...

514/514 [==============================] - 0s 331us/step - loss: 0.5328 - acc: 0.7140 - val_loss: 0.6672 - val_acc: 0.6969
Epoch 865/1000
514/514 [==============================] - 0s 303us/step - loss: 0.4979 - acc: 0.7451 - val_loss: 0.6724 - val_acc: 0.6890
Epoch 866/1000
514/514 [==============================] - 0s 342us/step - loss: 0.5128 - acc: 0.7451 - val_loss: 0.6686 - val_acc: 0.6929
Epoch 867/1000
514/514 [==============================] - 0s 338us/step - loss: 0.4763 - acc: 0.7588 - val_loss: 0.6661 - val_acc: 0.6850
Epoch 868/1000
514/514 [==============================] - 0s 321us/step - loss: 0.4710 - acc: 0.7607 - val_loss: 0.6882 - val_acc: 0.6890
Epoch 869/1000
514/514 [==============================] - 0s 333us/step - loss: 0.4743 - acc: 0.7763 - val_loss: 0.6925 - val_acc: 0.6850
Epoch 870/1000
514/514 [==============================] - 0s 325us/step - loss: 0.4754 - acc: 0.7802 - val_loss: 0.6874 - val_acc: 0.6969
Epoch 871/1000
514/514 [=========================

514/514 [==============================] - 0s 360us/step - loss: 0.4671 - acc: 0.7704 - val_loss: 0.6707 - val_acc: 0.6850
Epoch 897/1000
514/514 [==============================] - 0s 315us/step - loss: 0.4825 - acc: 0.7588 - val_loss: 0.6896 - val_acc: 0.6929
Epoch 898/1000
514/514 [==============================] - 0s 300us/step - loss: 0.4670 - acc: 0.7588 - val_loss: 0.6883 - val_acc: 0.6772
Epoch 899/1000
514/514 [==============================] - 0s 359us/step - loss: 0.4671 - acc: 0.7471 - val_loss: 0.6954 - val_acc: 0.7008
Epoch 900/1000
514/514 [==============================] - 0s 334us/step - loss: 0.4730 - acc: 0.7490 - val_loss: 0.6747 - val_acc: 0.6732
Epoch 901/1000
514/514 [==============================] - 0s 360us/step - loss: 0.4686 - acc: 0.7568 - val_loss: 0.6786 - val_acc: 0.6969
Epoch 902/1000
514/514 [==============================] - 0s 339us/step - loss: 0.4636 - acc: 0.7704 - val_loss: 0.6914 - val_acc: 0.6929
Epoch 903/1000
514/514 [=========================

514/514 [==============================] - 0s 335us/step - loss: 0.4577 - acc: 0.7704 - val_loss: 0.7129 - val_acc: 0.6732
Epoch 929/1000
514/514 [==============================] - 0s 336us/step - loss: 0.4571 - acc: 0.7763 - val_loss: 0.7246 - val_acc: 0.6811
Epoch 930/1000
514/514 [==============================] - 0s 328us/step - loss: 0.4567 - acc: 0.7724 - val_loss: 0.7201 - val_acc: 0.6772
Epoch 931/1000
514/514 [==============================] - 0s 352us/step - loss: 0.4527 - acc: 0.7860 - val_loss: 0.7214 - val_acc: 0.6811
Epoch 932/1000
514/514 [==============================] - 0s 348us/step - loss: 0.4549 - acc: 0.7704 - val_loss: 0.7262 - val_acc: 0.6654
Epoch 933/1000
514/514 [==============================] - 0s 313us/step - loss: 0.4698 - acc: 0.7665 - val_loss: 0.7211 - val_acc: 0.6693
Epoch 934/1000
514/514 [==============================] - 0s 306us/step - loss: 0.4650 - acc: 0.7646 - val_loss: 0.7220 - val_acc: 0.6457
Epoch 935/1000
514/514 [=========================

514/514 [==============================] - 0s 345us/step - loss: 0.4596 - acc: 0.7665 - val_loss: 0.7249 - val_acc: 0.6811
Epoch 961/1000
514/514 [==============================] - 0s 326us/step - loss: 0.4583 - acc: 0.7588 - val_loss: 0.7323 - val_acc: 0.6496
Epoch 962/1000
514/514 [==============================] - 0s 305us/step - loss: 0.4539 - acc: 0.7626 - val_loss: 0.7454 - val_acc: 0.6378
Epoch 963/1000
514/514 [==============================] - 0s 315us/step - loss: 0.4530 - acc: 0.7568 - val_loss: 0.7558 - val_acc: 0.6575
Epoch 964/1000
514/514 [==============================] - 0s 313us/step - loss: 0.4494 - acc: 0.7704 - val_loss: 0.7308 - val_acc: 0.6575
Epoch 965/1000
514/514 [==============================] - 0s 346us/step - loss: 0.4389 - acc: 0.7782 - val_loss: 0.7515 - val_acc: 0.6850
Epoch 966/1000
514/514 [==============================] - 0s 335us/step - loss: 0.4431 - acc: 0.7743 - val_loss: 0.7348 - val_acc: 0.6614
Epoch 967/1000
514/514 [=========================

514/514 [==============================] - 0s 328us/step - loss: 0.4363 - acc: 0.7782 - val_loss: 0.7742 - val_acc: 0.6850
Epoch 993/1000
514/514 [==============================] - 0s 329us/step - loss: 0.4501 - acc: 0.7879 - val_loss: 0.7718 - val_acc: 0.6575
Epoch 994/1000
514/514 [==============================] - 0s 316us/step - loss: 0.4702 - acc: 0.7471 - val_loss: 0.7905 - val_acc: 0.6181
Epoch 995/1000
514/514 [==============================] - 0s 334us/step - loss: 0.4629 - acc: 0.7549 - val_loss: 0.7514 - val_acc: 0.6260
Epoch 996/1000
514/514 [==============================] - 0s 333us/step - loss: 0.4445 - acc: 0.7724 - val_loss: 0.7891 - val_acc: 0.6535
Epoch 997/1000
514/514 [==============================] - 0s 310us/step - loss: 0.4305 - acc: 0.7899 - val_loss: 0.7894 - val_acc: 0.6732
Epoch 998/1000
514/514 [==============================] - 0s 311us/step - loss: 0.4270 - acc: 0.7860 - val_loss: 0.7865 - val_acc: 0.6614
Epoch 999/1000
514/514 [=========================